<a href="https://colab.research.google.com/github/vivek-varma/Volatality_Prediction_ML/blob/main/Regime_predcition_data_acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!pip -q install -U databento

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 3.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
notebook 6.5.7 requires jupyter-client<8,>=5.3.4, but you have jupyter-client 8.6.3 which is incompatible.
jupyter-kernel-gateway 2.5.2 requires jupyter-client<8.0,>=5.2.0, but you have jupyter-client 8.6.3 which is incompatible.


In [26]:
import os

api_key_path = '/content/drive/MyDrive/Regime_pred/api.txt'
with open(api_key_path, 'r') as f:
    api_key = f.read().strip()

print("API key extracted successfully.")

API key extracted successfully.


--2025-09-24 00:04:15--  ftp://ftp.databento.com/P4EGDVHF/GLBX-20250923-9B7BSXHP6T
           => ‘/content/drive/MyDrive/Regime_pred/Data/ES_Fut_data’
Resolving ftp.databento.com (ftp.databento.com)... 209.127.152.43
Connecting to ftp.databento.com (ftp.databento.com)|209.127.152.43|:21... connected.
Logging in as vivek.phone7@gmail.com ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /P4EGDVHF ... done.
==> SIZE GLBX-20250923-9B7BSXHP6T ... done.

==> PASV ... done.    ==> RETR GLBX-20250923-9B7BSXHP6T ... 
No such file ‘GLBX-20250923-9B7BSXHP6T’.



In [60]:
import os
import pandas as pd
import databento as db


SAVE_DIR = "/content/drive/MyDrive/Regime_pred/Data"
os.makedirs(SAVE_DIR, exist_ok=True)

client = db.Historical()

def save_year_es_1m(year: int):
    start = f"{year}-01-01T00:00"
    end   = f"{year+1}-01-01T00:00"
    print(f"\nDownloading {start} → {end}")

    resp = client.timeseries.get_range(
        dataset="GLBX.MDP3",
        symbols="ES.FUT",
        stype_in="parent",
        schema="ohlcv-1m",
        start=start,
        end=end,
    )
    df = resp.to_df()
    if df is None or len(df) == 0:
        print(f"-> No data for {year}")
        return

    # --- make timestamp column ---
    # Case 1: a time column exists already
    for c in ["ts_event", "ts_recv", "ts", "timestamp"]:
        if c in df.columns:
            df["timestamp"] = pd.to_datetime(df[c], utc=True)
            break
    else:
        # Case 2: the index holds the time (your 2018 case)
        df = df.reset_index()                      # creates 'ts_event' column
        if "ts_event" in df.columns:
            df.rename(columns={"ts_event":"timestamp"}, inplace=True)
        elif "index" in df.columns:
            df.rename(columns={"index":"timestamp"}, inplace=True)
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True)

    # keep useful columns
    keep = ["timestamp","open","high","low","close","volume","symbol","instrument_id","publisher_id","rtype"]
    keep = [k for k in keep if k in df.columns]
    out = df[keep].copy()

    # sort + de-dup
    dedup_keys = ["timestamp"] + (["symbol"] if "symbol" in out.columns else [])
    out = out.sort_values(dedup_keys).drop_duplicates(subset=dedup_keys).reset_index(drop=True)

    out_path = f"{SAVE_DIR}/ES_1min_{year}.csv"
    out.to_csv(out_path, index=False)
    print(f"✅ Saved {len(out):,} rows → {out_path}")

# Run for 2018
save_year_es_1m(2024)



✅ Saved 568,171 rows → /content/drive/MyDrive/Regime_pred/Data/ES_1min_2024.csv


In [61]:
import glob

files = sorted(glob.glob(f"{SAVE_DIR}/ES_1min_*.csv"))
assert files, "No yearly files found."

dfs = [pd.read_csv(f, parse_dates=["timestamp"], low_memory=False) for f in files]
master = pd.concat(dfs, ignore_index=True)

dedup_keys = ["timestamp"] + (["symbol"] if "symbol" in master.columns else [])
master = master.sort_values(dedup_keys).drop_duplicates(subset=dedup_keys).reset_index(drop=True)

out_master = f"{SAVE_DIR}/ES_1min_MASTER_2018_2025.csv"
master.to_csv(out_master, index=False)
print(f"✅ Master rows: {len(master):,} → {out_master}")
print("Range:", master['timestamp'].min(), "→", master['timestamp'].max())

✅ Master rows: 3,935,019 → /content/drive/MyDrive/Regime_pred/Data/ES_1min_MASTER_2018_2025.csv
Range: 2018-01-01 23:00:00+00:00 → 2024-12-31 21:59:00+00:00


In [63]:
MASTER_PATH = "/content/drive/MyDrive/Regime_pred/Data/ES_1min_MASTER_2018_2025.csv"
OUT_MASTER = "/content/drive/MyDrive/Regime_pred/Data/ES_1min_MASTER_continuous.csv"
OUT_MASTER_RTH = "/content/drive/MyDrive/Regime_pred/Data/ES_1min_MASTER_continuous_RTH.csv"

# 1) Load the master file
df = pd.read_csv(MASTER_PATH, low_memory=False, parse_dates=["timestamp"])
print("Loaded:", len(df), "rows | Columns:", df.columns.tolist())

# 2) Drop spreads and non-ES instruments
if "symbol" in df.columns:
    mask_es = df["symbol"].astype(str).str.startswith("ES")
    mask_mes = df["symbol"].astype(str).str.startswith("MES")  # micro ES
    mask_spread = df["symbol"].astype(str).str.contains("-")   # calendar spreads
    df = df[mask_es & ~mask_mes & ~mask_spread].copy()

# 3) Sort and deduplicate by timestamp+volume
df = df.sort_values(["timestamp","volume"], ascending=[True,False]).reset_index(drop=True)

# 4) Pick the most active contract per minute (max volume)
cols_core = ["timestamp","open","high","low","close","volume"]
if "symbol" in df.columns:
    cols_core += ["symbol"]
df_cont = df.groupby("timestamp", as_index=False).first()[cols_core]

# 5) (Optional) RTH filter (08:30–15:00 CT)
def rth_filter(data):
    data = data.copy()
    ts_ct = data["timestamp"].dt.tz_convert("America/Chicago")
    mask = (ts_ct.dt.time >= pd.to_datetime("08:30").time()) & \
           (ts_ct.dt.time <= pd.to_datetime("15:00").time())
    return data.loc[mask].reset_index(drop=True)

df_rth = rth_filter(df_cont)

# 6) Save final files
df_cont.to_csv(OUT_MASTER, index=False)
df_rth.to_csv(OUT_MASTER_RTH, index=False)

print(f"✅ Saved continuous full session → {OUT_MASTER} ({len(df_cont):,} rows)")
print(f"✅ Saved continuous RTH only     → {OUT_MASTER_RTH} ({len(df_rth):,} rows)")
print("Time range:", df_cont['timestamp'].min(), "→", df_cont['timestamp'].max())

Loaded: 3935019 rows | Columns: ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'symbol', 'instrument_id', 'publisher_id', 'rtype']
✅ Saved continuous full session → /content/drive/MyDrive/Regime_pred/Data/ES_1min_MASTER_continuous.csv (2,464,266 rows)
✅ Saved continuous RTH only     → /content/drive/MyDrive/Regime_pred/Data/ES_1min_MASTER_continuous_RTH.csv (694,805 rows)
Time range: 2018-01-01 23:00:00+00:00 → 2024-12-31 21:59:00+00:00


In [65]:
df = pd.read_csv("/content/drive/MyDrive/Regime_pred/Data/ES_1min_MASTER_continuous.csv")
df.head()

,timestamp,open,high,low,close,volume,symbol
0,2018-01-01 23:00:00+00:00,2675.25,2677.00,2674.5,2676.75,2939,ESH8
1,2018-01-01 23:01:00+00:00,2676.75,2678.00,2676.5,2678.00,1258,ESH8
2,2018-01-01 23:02:00+00:00,2678.00,2678.75,2677.5,2678.50,852,ESH8
3,2018-01-01 23:03:00+00:00,2678.50,2678.75,2678.0,2678.25,547,ESH8
4,2018-01-01 23:04:00+00:00,2678.25,2678.50,2678.0,2678.50,275,ESH8


In [66]:
# VIX (^VIX) daily close
!pip -q install yfinance
import yfinance as yf, pandas as pd, os
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

SAVE_DIR = "/content/drive/MyDrive/Regime_pred/Data"
os.makedirs(SAVE_DIR, exist_ok=True)

vix = yf.download("^VIX", start="2010-01-01")  # plenty of history
vix = vix.reset_index()[["Date","Close"]].rename(columns={"Date":"date","Close":"vix"})
vix["date"] = pd.to_datetime(vix["date"]).dt.tz_localize(None)
vix.to_csv(f"{SAVE_DIR}/VIX_daily.csv", index=False)
print("Saved:", len(vix), "rows →", f"{SAVE_DIR}/VIX_daily.csv")
vix.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-1553360886.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  vix = yf.download("^VIX", start="2010-01-01")  # plenty of history
[*********************100%***********************]  1 of 1 completed

Saved: 3955 rows → /content/drive/MyDrive/Regime_pred/Data/VIX_daily.csv


Price,date,vix
Ticker,,^VIX
0,2010-01-04,20.040001
1,2010-01-05,19.350000
2,2010-01-06,19.160000
3,2010-01-07,19.059999
4,2010-01-08,18.129999


In [69]:
import pandas as pd, io, requests, os

SAVE_DIR = "/content/drive/MyDrive/Regime_pred/Data"
os.makedirs(SAVE_DIR, exist_ok=True)

url = "https://cdn.cboe.com/api/global/us_indices/daily_prices/VVIX_History.csv"
r = requests.get(url, timeout=30); r.raise_for_status()
vvix = pd.read_csv(io.StringIO(r.text))

# Normalize headers
vvix.columns = vvix.columns.str.strip().str.lower()
# Accept either 'vvix' or any close-like column
date_col = next((c for c in vvix.columns if "date" in c), None)
if "vvix" in vvix.columns:
    val_col = "vvix"
else:
    val_col = next((c for c in vvix.columns if "close" in c or "px_last" in c), None)

if not date_col or not val_col:
    raise ValueError(f"Could not detect date/value columns. Columns: {vvix.columns.tolist()}")

vvix = vvix.rename(columns={date_col: "date", val_col: "vvix"})
vvix["date"] = pd.to_datetime(vvix["date"]).dt.tz_localize(None)
vvix = vvix[["date","vvix"]].dropna()

out_path = f"{SAVE_DIR}/VVIX_daily.csv"
vvix.to_csv(out_path, index=False)
print(f"✅ Saved {len(vvix):,} rows → {out_path}")
vvix.head()


✅ Saved 4,859 rows → /content/drive/MyDrive/Regime_pred/Data/VVIX_daily.csv


,date,vvix
0,2006-03-06,71.73
1,2006-03-15,15.71
2,2006-03-16,27.94
3,2006-03-17,28.60
4,2006-03-20,37.93


In [75]:
!pip -q install yfinance
import yfinance as yf
import pandas as pd, os
from google.colab import drive

drive.mount('/content/drive', force_remount=False)

SAVE_DIR = "/content/drive/MyDrive/Regime_pred/Data"
os.makedirs(SAVE_DIR, exist_ok=True)

# Yahoo tickers
tickers = ["^VIX", "^VIX3M", "^VIX6M"]

# Download daily closes
df = yf.download(tickers, start="2010-01-01")["Close"]
df = df.reset_index().rename(
    columns={"Date":"date", "^VIX":"VIX", "^VIX3M":"VIX3M", "^VIX6M":"VIX6M"}
)

df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
df = df.dropna().sort_values("date").reset_index(drop=True)

# Save
out_path = f"{SAVE_DIR}/VIX_term_stack.csv"
df.to_csv(out_path, index=False)
print(f"✅ Saved {len(df)} rows → {out_path}")
df.tail()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-212571394.py:15: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(tickers, start="2010-01-01")["Close"]
[*********************100%***********************]  3 of 3 completed


✅ Saved 3955 rows → /content/drive/MyDrive/Regime_pred/Data/VIX_term_stack.csv


Ticker,date,VIX,VIX3M,VIX6M
3950,2025-09-17,15.720000,18.440001,20.660000
3951,2025-09-18,15.700000,18.520000,20.730000
3952,2025-09-19,15.450000,18.480000,20.740000
3953,2025-09-22,16.100000,18.950001,21.100000
3954,2025-09-23,16.639999,19.440001,21.450001
